In [67]:
import pandas as pd
from rake_nltk import Rake
import operator
import os
import re
import json
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()

# var
KEYWORD_TRESH = 5

In [68]:
def clean(txt):
    txt=re.sub(r'\n',' ',txt)
    txt=re.sub(' +', ' ', txt)
    txt=re.sub(',', ' ', txt)
    txt=re.sub(r'\([^()]*\)','',txt)
    txt=re.sub(r'https?:\S+\sdoi','',txt)
    return txt.lower()

In [69]:
files =[]
task_directory = '../tasks/'
for (dirpath, dirnames, filenames) in os.walk(task_directory):
    filenames = [names for names in filenames if '.txt' in names]
    if filenames != []:
        files.append({'dirpath':dirpath, 'filenames':filenames})

In [72]:
key_phrases_dict = {}
key_phrases_dict['covid19'] = {'task_name':'covid19',
                                            'keywords':['covid-19','covid19','covid 19']}
key_phrases_dict['sars'] = {'task_name':'sars',
                                            'keywords':['sars-cov-2','cov2','sars']}

for file in filenames:
    with open(task_directory  + file, "r") as read_file:
        data = read_file.read()
    data = lemmatizer.lemmatize(data)
    r = Rake()
    r.extract_keywords_from_text(clean(data))
    keyphrases = r.get_ranked_phrases_with_scores()
    keyphrases = [k[1] for k in keyphrases if k[0] > KEYWORD_TRESH]
    
    key_phrases_dict[file.split(' ')[0]] = {'task_name':file,
                                            'keywords':keyphrases}
    
    
    

In [73]:
key_phrases_dict

{'covid19': {'task_name': 'covid19',
  'keywords': ['covid-19', 'covid19', 'covid 19']},
 'sars': {'task_name': 'sars', 'keywords': ['sars-cov-2', 'cov2', 'sars']},
 'Task_1': {'task_name': 'Task_1 What is known about transmission, incubation, and environmental stability.txt',
  'keywords': ['improve clinical processes disease models including animal models',
   'personal protective equipment',
   'monitor phenotypic change',
   'infected person implementation',
   'community settings role',
   'community settings effectiveness',
   'prevent secondary transmission',
   'virus immune response',
   'movement control strategies',
   'natural history transmission',
   'infection disease']},
 'Task_2': {'task_name': 'Task_2 What do we know about COVID-19 risk factors.txt',
  'keywords': ['basic reproductive number incubation period serial interval modes',
   'populations public health mitigation measures',
   'fatality among symptomatic hospitalized patients',
   'potential risks factors sm

In [74]:
with open(task_directory  + 'keywords.json', "w") as write_file:
    json.dump(key_phrases_dict, write_file)